# Pytorch tutorial 2. Neural Networks
## Replication of pytorch tutorial
##https://pytorch.org/tutorials/index.html

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        conv_maxpool1 = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        conv_maxpool2 = F.max_pool2d(F.relu(self.conv2(conv_maxpool1)), 2)
        conv_out = conv_maxpool2.view(-1, self.num_flat_features(conv_maxpool2))
        fc_relu1 = F.relu(self.fc1(conv_out))
        fc_relu2 = F.relu(self.fc2(fc_relu1))
        output = self.fc3(fc_relu2)
        return output
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [21]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [22]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([16, 1, 5, 5])


In [48]:
input_x = Variable(torch.randn(1,1,32,32))
out = net(input_x)
print(out)

Variable containing:
 0.1417  0.0952 -0.0699 -0.0091  0.0979  0.0524 -0.0527  0.1440  0.1527  0.0957
[torch.FloatTensor of size 1x10]



In [35]:
net.zero_grad()
out.backward(torch.randn(1,10))

##Loss Function

In [45]:
output = net(Variable(input_x))
target = Variable(torch.arange(1,11))
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.4453
[torch.FloatTensor of size 1]



##Backprop

In [46]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 16]

conv1.bias.grad after backward
Variable containing:
-0.0236
 0.0893
-0.0363
-0.1106
 0.0052
-0.0234
 0.0845
 0.0077
-0.0619
-0.0251
-0.0640
-0.0065
-0.0376
 0.0547
-0.0025
 0.0399
[torch.FloatTensor of size 16]



##Update the weights
weight = weight - learning_rate * gradient

In [47]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

In [55]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)

optimizer.zero_grad()
output = net(input_x)
loss = criterion(output, target)
loss.backward()
optimizer.step()



In [56]:
print(loss)
loss = criterion(output, target)
print(loss)

Variable containing:
 24.4282
[torch.FloatTensor of size 1]

Variable containing:
 24.4282
[torch.FloatTensor of size 1]

